In [1]:
import glob
import numpy as np
import pickle
from scipy import ndimage as ndimage
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

ModuleNotFoundError: No module named 'scipy'

In [ ]:
def resize_gestures(input_gestures, final_length=100):

    output_gestures = np.array([np.array([ndimage.zoom(x_i.T[j], final_length / len(x_i), mode='reflect') for j in range(np.size(x_i, 1))]).T for x_i in input_gestures])
    return output_gestures

In [ ]:
def load_gestures(root, resize_gesture_to_length=100):
   
  
    pattern = root + '/DHG/DHG2016/gesture_*/finger_*/subject_*/essai_*/skeleton_image.txt'

    gestures_filenames = sorted(glob.glob(pattern))
   
    gestures = [np.genfromtxt(f) for f in gestures_filenames]
    if resize_gesture_to_length is not None:
        gestures = resize_gestures(gestures, final_length=resize_gesture_to_length)

    labels_14 = [int(filename.split('/')[-5].split('_')[1]) for filename in gestures_filenames]

    return gestures, labels_14

In [2]:
os.path.abspath(os.getcwd())

NameError: name 'os' is not defined

In [3]:
gestures, labels_14 = load_gestures(root=os.path.abspath(os.getcwd()),
                                     resize_gesture_to_length=100)

NameError: name 'load_gestures' is not defined

In [4]:
gestures = np.array(gestures)

NameError: name 'gestures' is not defined

In [5]:
X_train, X_test, y_train, y_test = train_test_split(gestures,labels_14, test_size=0.3 )

NameError: name 'train_test_split' is not defined

In [6]:
X_test.shape

NameError: name 'X_test' is not defined

In [7]:
y_train = np.array(y_train)
y_test = np.array(y_test)

NameError: name 'y_train' is not defined

In [8]:
max(y_train)

NameError: name 'y_train' is not defined

In [9]:
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
y_train = to_categorical(y_train, 15)

NameError: name 'to_categorical' is not defined

In [11]:
y_test = to_categorical(y_test, 15)

NameError: name 'to_categorical' is not defined

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dense,Flatten, TimeDistributed,Dropout
import keras.layers

ModuleNotFoundError: No module named 'keras'

In [13]:
import keras.backend as K

ModuleNotFoundError: No module named 'keras'

In [14]:
img_size = X_train[0].shape
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0],1, 100, 44)
    X_test = X_test.reshape(X_test.shape[0],1, 100, 44)
else:
    X_train = X_train.reshape(X_train.shape[0], 100, 44, 1)
    X_test = X_test.reshape(X_test.shape[0], 100, 44, 1)

NameError: name 'X_train' is not defined

In [15]:
input_shape1 = X_train.shape[1:]
input_shape1

NameError: name 'X_train' is not defined

In [16]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [17]:
model = Sequential()
model.add(Conv2D(8, kernel_size=(3,3), strides = (1,1), padding='same', activation = 'relu',input_shape = input_shape1))
model.add(Conv2D(8, kernel_size=(3,3), strides = (1,1), padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, kernel_size=(3,3), strides = (1,1), padding='same', activation = 'relu'))
model.add(Conv2D(16, kernel_size=(3,3), strides = (1,1), padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), strides = (1,1), padding='same', activation = 'relu'))
model.add(Conv2D(32, kernel_size=(3,3), strides = (1,1), padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.5))
model.add(TimeDistributed(Flatten()))

model.add(tf.keras.layers.LSTM(units =  128 ,activation = 'relu'))



model.add(Dense(units= 128, activation = 'relu'))

model.add(Dropout(0.2))
model.add(Dense(units= 128, activation = 'relu'))

model.add(Dropout(0.2))
model.add(Dense(units= 15, activation = 'softmax'))



#model.summary()



NameError: name 'Sequential' is not defined

In [18]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

NameError: name 'model' is not defined

In [19]:
history = model.fit(X_train, y_train, batch_size= 64,epochs=100, validation_split=0.2)

NameError: name 'model' is not defined

In [20]:
plt.plot(history.epoch, history.history['loss'])
plt.plot(history.epoch, history.history['val_loss'])

NameError: name 'plt' is not defined

In [21]:
plt.plot(history.epoch, history.history['accuracy'])
plt.plot(history.epoch, history.history['val_accuracy'])

NameError: name 'plt' is not defined

In [22]:
model.evaluate(X_test, y_test)

NameError: name 'model' is not defined

In [23]:
from sklearn.metrics import confusion_matrix,classification_report

ModuleNotFoundError: No module named 'sklearn'

In [24]:
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(model.predict(X_test), axis = 1)

confusion_matrix(y_true, y_pred)

NameError: name 'y_test' is not defined

In [25]:
print(classification_report(y_true, y_pred))

NameError: name 'classification_report' is not defined